In [75]:
%matplotlib inline
%matplotlib nbagg
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [76]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 2
img_channels = 3

In [77]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [78]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
    print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
    print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
print test_labels[0:10, :]
test_dataset_info = np.sum(test_labels[:, :], axis = 0)
print("Test Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(test_dataset_info[0], test_dataset_info[1]))
train_dataset_info = np.sum(train_labels[:, :], axis = 0)
print("Train Dataset Info : [1, 0] = {0}, [0, 1] = {1}".format(train_dataset_info[0], train_dataset_info[1]))

Training Set (648, 70, 100, 3) (648, 2)
Test Set (200, 70, 100, 3) (200, 2)
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
Test Dataset Info : [1, 0] = 149.0, [0, 1] = 51.0
Train Dataset Info : [1, 0] = 331.0, [0, 1] = 317.0


In [79]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [80]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 8, depth = 32, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 4).\
        conv2d(kernel = 4, depth = 64, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        conv2d(kernel = 3, depth = 64, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 1).\
        conv2d(kernel = 2, depth = 128, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 1).\
        conv2d(kernel = 2, depth = 128, name = "conv_layer_3").\
        max_pool(kernel = 2, stride = 1).\
        flatten().\
        fully_connected(size = 1024, name = "fc_layer_1", activation_fn = tf.nn.relu).\
        fully_connected(size = 512, name = "fc_layer_2",  activation_fn = None).\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [81]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-4).minimize(loss)

In [82]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [83]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [84]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [85]:
train_batch_size = 128

In [86]:
num_steps = 10
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print("Variable Initialized")
    for step in range(num_steps):
        offset = (step * train_batch_size) % (train_labels.shape[0] - train_batch_size)
        print offset
        batch_train = train_dataset[offset:(offset + train_batch_size), :]
        batch_labels = train_labels[offset:(offset + train_batch_size), :]
        feed_dict_train = {
            input_layer: batch_train,
            y_true: batch_labels
        }
        
        _, l = sess.run([optimizer, loss], feed_dict = feed_dict_train)
        print("Loss : {0}".format(l))
        
    feed_dict_test = {
        input_layer: test_dataset,
        y_true: test_labels
    }
    true_value_class, acc, pred_value, pred_value_class = sess.run([y_true_cls, accuracy, y_pred, y_pred_cls], feed_dict = feed_dict_test)
    print("Accuracy {:.2%}".format(acc))
    print("True Value Class")
    print true_value_class[0:10]
    print("Pred Value")
    print pred_value[0:20]
    print("Pred Value Class")
    print pred_value_class

Variable Initialized
0
Loss : 0.714734494686
128
Loss : 0.659823656082
256
Loss : 0.582350373268
384
Loss : 0.809815227985
512
Loss : 0.736865878105
120
Loss : 0.679297327995
248
Loss : 0.620877146721
376
Loss : 0.744383573532
504
Loss : 0.677604079247
112
Loss : 0.693195700645
Accuracy 60.50%
True Value Class
[0 0 1 0 1 0 0 0 0 0]
Pred Value
[[ 0.55968082  0.44031918]
 [ 0.54518092  0.45481914]
 [ 0.62046927  0.37953073]
 [ 0.56727356  0.43272647]
 [ 0.55781925  0.44218075]
 [ 0.53344387  0.46655607]
 [ 0.52727211  0.47272792]
 [ 0.50901663  0.49098334]
 [ 0.53077811  0.46922192]
 [ 0.52709639  0.47290358]
 [ 0.57243323  0.4275668 ]
 [ 0.52535057  0.47464946]
 [ 0.54822767  0.4517723 ]
 [ 0.49478802  0.50521201]
 [ 0.52680731  0.47319269]
 [ 0.57728481  0.42271519]
 [ 0.49117598  0.50882405]
 [ 0.5114485   0.48855141]
 [ 0.48555431  0.51444566]
 [ 0.51263416  0.48736587]]
Pred Value Class
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0